In [3]:
import flyte
import flyte.git
import flyte.storage

In [4]:
flyte.init_from_config(flyte.git.config_from_root(), storage=flyte.storage.S3.auto(region="us-east-2"))

In [5]:
env = flyte.TaskEnvironment(name="pd_data", image=flyte.Image.from_debian_base().with_pip_packages("pandas", "pyarrow"))

In [6]:
import pandas as pd

BASIC_EMPLOYEE_DATA = {
    "employee_id": range(1001, 1009),
    "name": ["Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah"],
    "department": ["HR", "Engineering", "Engineering", "Marketing", "Finance", "Finance", "HR", "Engineering"],
    "hire_date": pd.to_datetime(
        ["2018-01-15", "2019-03-22", "2020-07-10", "2017-11-01", "2021-06-05", "2018-09-13", "2022-01-07", "2020-12-30"]
    ),
}

In [7]:
@env.task
async def create_raw_dataframe() -> pd.DataFrame:
    """
    This task creates a raw pandas DataFrame with basic employee information.
    This is the most basic use-case of how to pass dataframes (of all kinds, not just pandas). Create the dataframe
    as normal, and return it. Note that the output signature is of the dataframe library type.
    Uploading of the actual bits of the dataframe (which for pandas is serialized to parquet) happens at the
    end of the task, the TypeEngine uploads from memory to blob store.
    """
    return pd.DataFrame(BASIC_EMPLOYEE_DATA)

In [8]:
r = flyte.run(create_raw_dataframe)

16:12:15.017844 WARNING  remote_builder.py:86 -  Image 356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo:flyte-5415240d92c36ab6a384a8ce6884c49c found.    
                         Skip building.

16:12:15.029816 WARNING  _deploy.py:160 -  Built Image for environment pd_data, image:                                                                          
                         356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo:flyte-5415240d92c36ab6a384a8ce6884c49c

In [9]:
r.url

'https://demo.hosted.unionai.cloud/v2/runs/project/ketan/domain/development/rs8bvvlqq5sx58mm9hkc'

In [10]:
r.wait()

Output()

Run 'rs8bvvlqq5sx58mm9hkc' completed successfully.

In [11]:
r.inputs()

{}

In [12]:
outputs = r.outputs()

In [13]:
outputs.pb2

literals {
  name: "o0"
  value {
    scalar {
      structured_dataset {
        uri: "s3://union-oc-production-demo/metadata/v2/demo/ketan/development/rs8bvvlqq5sx58mm9hkc/a0/1/vn/rs8bvvlqq5sx58mm9hkc-a0-0/6d95a13886556901bc810c4c942025b0"
        metadata {
          structured_dataset_type {
            format: "parquet"
          }
        }
      }
    }
  }
}

In [14]:
df = outputs[0]

In [15]:
await df.open(pd.DataFrame).all()

employee_id     name   department  hire_date
0         1001    Alice           HR 2018-01-15
1         1002      Bob  Engineering 2019-03-22
2         1003  Charlie  Engineering 2020-07-10
3         1004    Diana    Marketing 2017-11-01
4         1005    Ethan      Finance 2021-06-05
5         1006    Fiona      Finance 2018-09-13
6         1007   George           HR 2022-01-07
7         1008   Hannah  Engineering 2020-12-30